In [ ]:
%config Completer.use_jedi = False

In [ ]:
# Add project src to path.
import set_path

# Import modules.
import numpy as np
import time

%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import src.fusion as fusion
import src.utils.las_utils as las_utils
import src.utils.ahn_utils as ahn_utils
import src.utils.clip_utils as clip_utils
from src.utils.labels import Labels

## Set-up

In [ ]:
# All possible parameters:
# params = {'r_mult': 1.5,
#           'search_pad': 1.5,
#           'max_dist': 1.2,
#           'voxel_res': 0.2,
#           'seed_height': 1.75,
#           'min_height': 2.,
#           'max_r': 0.5,
#           'min_points': 500,
#           'z_min': 0.2,
#           'z_max': 2.7,
#           'label_height': 4.}

tree_params = {'seed_height': 1.75, 'min_points': 500, 'max_r': 0.5, 'label_height': 5.}
light_params = {'seed_height': 2.25, 'min_points': 400, 'max_r': 0.2, 'label_height': 5.}
sign_params = {'seed_height': 1.75, 'min_points': 200, 'max_r': 0.2, 'min_height': 1.2, 'z_max': 2., 'label_height': 3.}

In [ ]:
# Data folder for the BGT fuser.
#bgt_data_file = '../datasets/bgt/bgt_points.csv'
bgt_data_file = '../datasets/bgt/custom_points_demo.csv'
#tile_code = '2386_9702'
tile_code = '2397_9705'

# Load elevation data.
ahn_data_folder = '../datasets/ahn/'
ahn_reader = ahn_utils.NPZReader(data_folder=ahn_data_folder)

# Fusers for BGT point data.
tree_fuser = fusion.BGTPointFuser(Labels.TREE, bgt_type='boom', bgt_file=bgt_data_file, ahn_reader=ahn_reader, params=tree_params)
light_fuser = fusion.BGTPointFuser(Labels.STREET_LIGHT, bgt_type='lichtmast', bgt_file=bgt_data_file, ahn_reader=ahn_reader, params=light_params)
sign_fuser = fusion.BGTPointFuser(Labels.TRAFFIC_SIGN, bgt_type='verkeersbord', bgt_file=bgt_data_file, ahn_reader=ahn_reader, params=sign_params)

In [ ]:
# Load labeled and grown LAS file.
las_file = '../datasets/pointcloud/grown_' + tile_code + '.laz'
las = las_utils.read_las(las_file)

labels = las.label
points = np.vstack((las.x, las.y, las.z)).T

## Match point objects with clusters

In [ ]:
mask = (labels != Labels.GROUND) & (labels != Labels.BUILDING) & (labels != Labels.CAR)

start = time.time()
tree_mask = tree_fuser.get_label_mask(tile_code, points, mask)
end = time.time()
print(f'Trees fused in {end-start:.2f} seconds.\n')

start = time.time()
mask[tree_mask] = False
light_mask = light_fuser.get_label_mask(tile_code, points, mask)
end = time.time()
print(f'Street lights fused in {end-start:.2f} seconds.\n')

start = time.time()
mask[light_mask] = False
sign_mask = sign_fuser.get_label_mask(tile_code, points, mask)
end = time.time()
print(f'Traffic signs fused in {end-start:.2f} seconds.')

In [ ]:
labels[tree_mask] = tree_fuser.get_label()
labels[light_mask] = light_fuser.get_label()
labels[sign_mask] = sign_fuser.get_label()

In [ ]:
# Cluster based region growing.
# Not for trees yet.
from src.region_growing import LabelConnectedComp

ahn_tile = ahn_reader.filter_tile(tile_code)
avg_ground_height = np.nanmean(ahn_tile['ground_surface'])
excl_labels = (Labels.GROUND, Labels.BUILDING, Labels.TREE)

start = time.time()

plane_height = 3.25
height_mask_ids = np.where(points[:,2] > avg_ground_height + plane_height)[0]
lcc_light = LabelConnectedComp(Labels.STREET_LIGHT, exclude_labels=excl_labels,
                               octree_level=9, min_component_size=100, threshold=0.05)
lcc_light_mask = lcc_light.get_label_mask(points=points[height_mask_ids], labels=labels[height_mask_ids])
labels[height_mask_ids[lcc_light_mask]] = lcc_light.get_label()

plane_height = 1.5
height_mask_ids = np.where(points[:,2] <= avg_ground_height + plane_height)[0]
lcc_light = LabelConnectedComp(Labels.STREET_LIGHT, exclude_labels=excl_labels,
                               octree_level=9, min_component_size=100, threshold=0.5)
lcc_light_mask = lcc_light.get_label_mask(points=points[height_mask_ids], labels=labels[height_mask_ids])
labels[height_mask_ids[lcc_light_mask]] = lcc_light.get_label()

end = time.time()
print(f'Street lights grown in {end-start:.2f} seconds.\n')

start = time.time()

plane_height = 1.5
height_mask_ids = np.where(points[:,2] > avg_ground_height + plane_height)[0]
lcc_sign = LabelConnectedComp(Labels.TRAFFIC_SIGN, exclude_labels=excl_labels,
                              octree_level=9, min_component_size=100, threshold=0.05)
lcc_sign_mask = lcc_sign.get_label_mask(points=points[height_mask_ids], labels=labels[height_mask_ids])
labels[height_mask_ids[lcc_sign_mask]] = lcc_sign.get_label()

plane_height = 1.5
height_mask_ids = np.where(points[:,2] <= avg_ground_height + plane_height)[0]
lcc_sign = LabelConnectedComp(Labels.TRAFFIC_SIGN, exclude_labels=excl_labels,
                              octree_level=9, min_component_size=100, threshold=0.5)
lcc_sign_mask = lcc_sign.get_label_mask(points=points[height_mask_ids], labels=labels[height_mask_ids])
labels[height_mask_ids[lcc_sign_mask]] = lcc_sign.get_label()

end = time.time()
print(f'Traffic signs grown in {end-start:.2f} seconds.')

In [ ]:
# Save the result.
out_file = '../datasets/pointcloud/poles_custom_' + tile_code + '.laz'
las_utils.label_and_save_las(las, labels, out_file)

## Visualize results

In [ ]:
# Visualize the resulting match.
((x_min, y_max), (x_max, y_min)) = las_utils.get_bbox_from_tile_code(tile_code)
viz_mask = clip_utils.box_clip(points, np.array([x_min, y_min, x_max, y_max]), bottom=avg_ground_height+1.7, top=avg_ground_height+1.8)
label_set = np.unique(labels[viz_mask])

fig, ax = plt.subplots(1, constrained_layout=True)

for label in label_set:
    label_mask = viz_mask & (labels == label)
    scatter = ax.scatter(points[label_mask, 0], points[label_mask, 1], marker='.', label=Labels.get_str(label))

box = patches.Rectangle((x_min, y_min), x_max-x_min, y_max-y_min, linewidth=1, linestyle='--', edgecolor='grey', fill=False)

ax.add_patch(box)
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.axis('equal')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()